# `mlfinlab.bet_sizing` - Module Tutorial
The following is a tutorial in how to apply the functions in the `mlfinlab.bet_sizing` module. The exercises from Chapter 10 are used as examples.

In [1]:
# imports
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns

from IPython.display import display
import datetime as dt

# mlfinlab imports
from mlfinlab.bet_sizing import bet_size_dynamic
from mlfinlab.bet_sizing.ch10_snippets import get_w, get_t_pos, limit_price, bet_size, inv_price
from mlfinlab.bet_sizing.ch10_snippets import get_w_power, get_t_pos_power, limit_price_power, bet_size_power, inv_price_power


----
#### Dynamic Bet Size example
Randomly generate some samples for current position, maximum position, market price, forecast price, and supply some calibration parameters. While these values may not be completely realistic (especially when taken as a time series), they test operation of the dynamic bet size functions.

In [2]:
# setup parameters
np.random.seed(10)
sample_size = 1_000

# assign dates for index
start_date = dt.datetime(2000, 1, 1)  # starting at 01-JAN-2000
date_step = dt.timedelta(days=1)
dates = np.array([start_date + i*date_step for i in range(sample_size)])

curr_pos = pd.Series(data=np.random.uniform(80, 130, sample_size).astype(int), index=dates)
#curr_pos = 100
max_pos = pd.Series(data=np.array([135 for i in range(sample_size)]).astype(int), index=dates)
#max_pos = 135
market_price = pd.Series(data=np.random.uniform(75, 155, sample_size), index=dates)
#market_price = 201.5
forecast_price = pd.Series(data=np.random.uniform(70, 160, sample_size), index=dates)
#forecast_price = 197.45
cal_div = 10
cal_bet_size = 0.95
bet_function = 'sigmoid'

df_input = pd.concat([curr_pos, max_pos, market_price, forecast_price], axis=1)
df_input = df_input.rename(columns={0: 'pos',  # current position
                                    1: 'max_pos',  # maximum position
                                    2: 'm_p',  # market price
                                    3: 'f'})  # forecast price

display(df_input.head(5))

df_bets = bet_size_dynamic(curr_pos, max_pos, market_price, forecast_price, cal_div, cal_bet_size, bet_function)
display(df_bets.head(5))

,pos,max_pos,m_p,f
2000-01-01,118,135,102.866394,123.079981
2000-01-02,81,135,139.717029,139.611124
2000-01-03,111,135,111.770975,120.262382
2000-01-04,117,135,125.089944,86.670071
2000-01-05,104,135,91.291746,103.070211


,bet_size,t_pos,l_p
2000-01-01,0.987036,133,120.213411
2000-01-02,-0.032204,-4,0.000000
2000-01-03,0.932574,125,117.726961
2000-01-04,-0.996361,-134,5.681977
2000-01-05,0.963200,130,100.570138


Combine the results into a single `DataFrame`. Note that some of the limit prices are either zero or `np.nan`, neither of which are realistic. The columns at the right end of the `DataFrame` explore some reasons as to why the limit prices may not be practical values.

In [3]:
df2 = pd.concat([df_input, df_bets], axis=1)
df2['sign(t_pos - pos)'] = df2.apply(lambda x: np.sign(x.t_pos - x.pos), axis=1)
df2['t_pos == pos'] = df2.apply(lambda x: x.t_pos == x.pos, axis = 1)
df2['sum can exec'] = df2.apply(lambda x: abs(x.t_pos) - abs(x.pos + np.sign(x.t_pos-x.pos)) > 0, axis=1)
df2['sign(m_p - pos)'] = df2.apply(lambda x: np.sign(x.m_p - x.pos), axis=1)
df2['is l_p sensible'] = df2.apply(lambda x: x.m_p < x.l_p < x.f if x.bet_size > 0 else np.nan, axis=1)
display(df2.head(35))

,pos,max_pos,m_p,f,bet_size,t_pos,l_p,sign(t_pos - pos),t_pos == pos,sum can exec,sign(m_p - pos),is l_p sensible
2000-01-01,118,135,102.866394,123.079981,0.987036,133,120.213411,1.0,False,True,-1.0,True
2000-01-02,81,135,139.717029,139.611124,-0.032204,-4,0.000000,-1.0,False,False,1.0,NaN
2000-01-03,111,135,111.770975,120.262382,0.932574,125,117.726961,1.0,False,True,1.0,True
2000-01-04,117,135,125.089944,86.670071,-0.996361,-134,5.681977,-1.0,False,True,1.0,NaN
2000-01-05,104,135,91.291746,103.070211,0.963200,130,100.570138,1.0,False,True,-1.0,True
2000-01-06,91,135,82.494032,132.101356,0.997812,134,129.770719,1.0,False,True,-1.0,True
2000-01-07,89,135,153.042077,139.583669,-0.971449,-131,26.237414,-1.0,False,True,1.0,NaN
2000-01-08,118,135,104.387506,101.421284,-0.669971,-90,0.000000,-1.0,False,False,-1.0,NaN
2000-01-09,88,135,121.102717,85.382205,-0.995793,-134,17.237153,-1.0,False,True,1.0,NaN
2000-01-10,84,135,94.674565,82.373162,-0.966108,-130,17.268069,-1.0,False,True,1.0,NaN


In [4]:
a, b, c, d = 1, 2, 3, 4
print(a,b,c,d)
for i in [a, b, c, d]:
    i = i*i
print(a,b,c,d)

1 2 3 4
1 2 3 4


In [5]:
a = True
print(a)

True


In [6]:
a = a and False
print(a)

False


In [7]:
curr_pos.size

1000

In [8]:
True and not False

True

In [9]:
d = {'a':1, 'b':2, 'c':3}
for i in d:
    print(i)

a
b
c


In [4]:
def square(n):
    return n*n

def cube(n):
    return n*n*n

def quart(n):
    return n*n*n*n

d_func = {'sq': square,
          'cu': cube,
          'qu': quart}

f = 'sq'
print({'sq': square, 'cu': cube, 'qu': quart}[f](4))

16


In [3]:
m = 0.7
x = 4
f = 110.6

#w_param = get_w_power(x, m)
#print(f"w_param: {w_param}")

#inverted_price = inv_price_power(f, w_param, m)
#print(f"inverted_price: {inverted_price}")

#lim_price = limit_price_power(120, 110, f, w_param, 150)
#print(f"limit_price: {lim_price}")

print(f"bet size: {bet_size_power(0.4, x)}")

AssertionError: Price divergence must be between -1 and 1, inclusive. Found price divergence value: 4

In [6]:
(-1 <= -0.3 <= 1)

True